In [3]:
import numpy as np
from single_shot import print_matrix
import single_shot

In [4]:
def calculate_gamma(ei_vec):
    Ne = 18
    N = 10
    gamma = np.zeros((N, N), dtype=np.float64)
    for k in range(int(Ne / 2)):  # go through all orbitals that are occupied!
        vec_i = ei_vec[:, k][np.newaxis]
        gamma += vec_i.T @ vec_i  #
        # for i in range(N):
        #     for j in range(N):
        #         gamma[i, j] += ei_vec[i, k] * ei_vec[j, k]
    return gamma

In [5]:
obj = single_shot.Householder(10, 18, 8)
obj.calculate_one()
print_matrix(obj.h)

calculation with Ns=10, Ne=18, density=1.8
   +0.000000     -1.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     -1.000000
   -1.000000     +0.000000     -1.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000
   +0.000000     -1.000000     +0.000000     -1.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000
   +0.000000     +0.000000     -1.000000     +0.000000     -1.000000     +0.000000     +0.000000     +0.000000     +0.000000     +0.000000
   +0.000000     +0.000000     +0.000000     -1.000000     +0.000000     -1.000000     +0.000000     +0.000000     +0.000000     +0.000000
   +0.000000     +0.000000     +0.000000     +0.000000     -1.000000     +0.000000     -1.000000     +0.000000     +0.000000     +0.000000
   +0.000000     +0.000000     +0.000000     +0.000000     +0.000000     -1.000000     +0.000000     -1.000000     +0.00000

# This is done by numpy

In [6]:
ei_val, ei_vec = np.linalg.eig(obj.h)  # v[:,i] corresponds to eigval w[i]
idx = ei_val.argsort()[::1] # Sorting matrix and vector by ascending order
ei_val = ei_val[idx]
ei_vec = ei_vec[:, idx]


In [7]:
# Here is gamma calculated by numpys eigenvalues
calculate_gamma(ei_vec)

array([[ 0.91512823,  0.06921725, -0.10220641,  0.08253605, -0.08022394,
         0.09111379, -0.11753138,  0.13692925, -0.11516649,  0.12020366],
       [ 0.06921725,  0.89779359,  0.08253605, -0.08022394,  0.09111379,
        -0.11753138,  0.13692925, -0.11516649,  0.12020366, -0.08487177],
       [-0.10220641,  0.08253605,  0.91977606,  0.09111379, -0.11753138,
         0.13692925, -0.11516649,  0.12020366, -0.08487177,  0.06921725],
       [ 0.08253605, -0.08022394,  0.09111379,  0.88246862,  0.13692925,
        -0.11516649,  0.12020366, -0.08487177,  0.06921725, -0.10220641],
       [-0.08022394,  0.09111379, -0.11753138,  0.13692925,  0.88483351,
         0.12020366, -0.08487177,  0.06921725, -0.10220641,  0.08253605],
       [ 0.09111379, -0.11753138,  0.13692925, -0.11516649,  0.12020366,
         0.91512823,  0.06921725, -0.10220641,  0.08253605, -0.08022394],
       [-0.11753138,  0.13692925, -0.11516649,  0.12020366, -0.08487177,
         0.06921725,  0.89779359,  0.08253605

# Test if ei_vec are really eigenvectors:
I calculated h * vec / ei_val - vec for each eigenvector:

In [8]:
for i in range(len(obj.ei_val)):
    print(i,np.sum(np.square(obj.h @ obj.ei_vec[:, i] / obj.ei_val[i] - obj.ei_vec[:, i])))

0 6.77927340424307e-32
1 1.0496318196910435e-31
2 9.976317111925882e-32
3 1.6524478922842484e-31
4 4.503064951220406e-32
5 9.282982331946477e-32
6 2.2400972665654037e-31
7 8.108165065870263e-32
8 4.126304905849653e-32
9 6.77927340424307e-32


They seem to be eigenvectors.
We can also try to see if they are linear combination of ones that are calculated by fortran:

In [9]:
fr = np.loadtxt('ei_vec_10_18.txt')

In [10]:
for i, j in ((1,2), (3,4), (5,6), (7,8)):
    print('vectors with indeces:', i, j)
    i_proj_i = np.dot(ei_vec[:,i], fr[:, i])
    i_proj_j = np.dot(ei_vec[:,i], fr[:, j])
    j_proj_i = np.dot(ei_vec[:,j], fr[:, i])
    j_proj_j = np.dot(ei_vec[:,j], fr[:, j])
    print(i_proj_i, i_proj_j, j_proj_i, j_proj_j)
    print('\tsums of squares:', i_proj_i ** 2 + i_proj_j  ** 2, j_proj_i ** 2 + j_proj_j  ** 2)
    print('\tDot product between i and j:', np.dot(ei_vec[:,i], ei_vec[:, j]))
    print('\tDot product between i and j in fr:', np.dot(fr[:,i], fr[:, j]))

vectors with indeces: 1 2
-0.4039706251354577 0.9147711638063419 -0.8945505873693391 -0.4469651959772277
	sums of squares: 0.9999985481039417 0.9999986397777911
	Dot product between i and j: -0.047499151907367934
	Dot product between i and j in fr: -2.0282399998405864e-07
vectors with indeces: 3 4
0.3664891527221191 -0.9304227066508538 -0.9132220678515395 -0.40746257796626434
	sums of squares: 1.0000007121144776 1.0000002976539557
	Dot product between i and j: 0.04442704210688851
	Dot product between i and j in fr: -5.773760000243877e-07
vectors with indeces: 5 6
0.98286237087748 0.1843409200006014 -0.24189464670341743 0.9703025006707027
	sums of squares: 1.000000014873569 0.9999999629115901
	Dot product between i and j: -0.0588830586875494
	Dot product between i and j in fr: 3.4629799997715205e-07
vectors with indeces: 7 8
0.8631794649343396 0.5048972205578104 -0.6022227863097909 0.798327958460435
	sums of squares: 0.9999999920113352 0.9999998136103341
	Dot product between i and j: -0

It looks like eigenvectors that are given by numpy with multiplicity >= 2 are not orthogonal (dot product is much higher than in fr and is
in same order of magnitude as numeric error of 1RDM)
Google said that if we want to make vectors orthogonal we should use
https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process

In [11]:
ei_vec_orth = ei_vec
for i, j in ((1,2), (3,4), (5,6), (7,8)):
    ei_vec[:, j] = ei_vec[:, j] - ei_vec[:, i] * (np.dot(ei_vec[:, j],ei_vec[:, i])/np.dot(ei_vec[:, i],ei_vec[:, i]))
    print(i_proj_i, i_proj_j, j_proj_i, j_proj_j)
    print('\tsums of squares:', i_proj_i ** 2 + i_proj_j  ** 2, j_proj_i ** 2 + j_proj_j  ** 2)
    print('\tDot product between i and j:', np.dot(ei_vec[:,i], ei_vec[:, j]))
    print('\tDot product between i and j in fr:', np.dot(fr[:,i], fr[:, j]))

0.8631794649343396 0.5048972205578104 -0.6022227863097909 0.798327958460435
	sums of squares: 0.9999999920113352 0.9999998136103341
	Dot product between i and j: 0.0
	Dot product between i and j in fr: -2.0282399998405864e-07
0.8631794649343396 0.5048972205578104 -0.6022227863097909 0.798327958460435
	sums of squares: 0.9999999920113352 0.9999998136103341
	Dot product between i and j: 6.938893903907228e-18
	Dot product between i and j in fr: -5.773760000243877e-07
0.8631794649343396 0.5048972205578104 -0.6022227863097909 0.798327958460435
	sums of squares: 0.9999999920113352 0.9999998136103341
	Dot product between i and j: 4.85722573273506e-17
	Dot product between i and j in fr: 3.4629799997715205e-07
0.8631794649343396 0.5048972205578104 -0.6022227863097909 0.798327958460435
	sums of squares: 0.9999999920113352 0.9999998136103341
	Dot product between i and j: 1.3877787807814457e-17
	Dot product between i and j in fr: 1.9562400000588953e-07


In [12]:
print_matrix(calculate_gamma(ei_vec_orth))
# orthogonalization didn't solve the problem

   +0.899156     +0.100071     -0.100043     +0.099599     -0.099443     +0.100844     -0.100071     +0.100043     -0.099599     +0.099443
   +0.100071     +0.897875     +0.101037     -0.100611     +0.101359     -0.100071     +0.099527     -0.098431     +0.098004     -0.098762
   -0.100043     +0.101037     +0.897307     +0.102798     -0.101239     +0.100043     -0.098431     +0.097489     -0.097594     +0.098633
   +0.099599     -0.100611     +0.102798     +0.896678     +0.101482     -0.099599     +0.098004     -0.097594     +0.098117     -0.098875
   -0.099443     +0.101359     -0.101239     +0.101482     +0.898319     +0.099443     -0.098762     +0.098633     -0.098875     +0.099083
   +0.100844     -0.100071     +0.100043     -0.099599     +0.099443     +0.899156     +0.100071     -0.100043     +0.099599     -0.099443
   -0.100071     +0.099527     -0.098431     +0.098004     -0.098762     +0.100071     +0.897875     +0.101037     -0.100611     +0.101359
   +0.100043     -0.098431 

In [13]:
from sympy import Matrix
M = Matrix(obj.h)
eig = M.eigenvects()

In [20]:
ei_vec_matrix = []
ei_val_matrix = []
for eigenval, multiplicity, vectors in eig:
    print(f'Eigenvalue {eigenval.evalf()} ({eigenval}), multiplicity {multiplicity}')
    ei_vec_matrix.append(list(vectors[0]))
    ei_val_matrix.append(eigenval)

ei_vec_matrix = np.array(ei_vec_matrix, dtype=np.float64).T
ei_val_matrix = np.array(ei_val_matrix, dtype=np.float64)
idx = ei_val_matrix.argsort()[::1] # Sorting matrix and vector by ascending order
ei_val_matrix = ei_val_matrix[idx]
ei_vec_matrix = ei_vec_matrix[:, idx]

print(ei_vec_matrix)

Eigenvalue -1.61803398874989 (-1.61803398874989), multiplicity 1
Eigenvalue -2.00000000000000 (-2.00000000000000), multiplicity 1
Eigenvalue -1.61803398874989 (-1.61803398874989), multiplicity 1
Eigenvalue -0.618033988749895 (-0.618033988749895), multiplicity 1
Eigenvalue -0.618033988749895 (-0.618033988749895), multiplicity 1
Eigenvalue 0.618033988749895 (0.618033988749895), multiplicity 1
Eigenvalue 0.618033988749895 (0.618033988749895), multiplicity 1
Eigenvalue 2.00000000000000 (2.00000000000000), multiplicity 1
Eigenvalue 1.61803398874989 (1.61803398874989), multiplicity 1
Eigenvalue 1.61803398874989 (1.61803398874989), multiplicity 1
[[ 0.31622777 -0.22687248  0.38067653  0.43567718  0.10982479  0.4468003
  -0.10858231 -0.398187    0.20358563 -0.31622777]
 [ 0.31622777 -0.41007286  0.16991071  0.23061487 -0.3784547  -0.15635033
  -0.38772227  0.44180469  0.06934421  0.31622777]
 [ 0.31622777 -0.43663935 -0.10575523 -0.29314935 -0.34372265 -0.35017048
   0.34820785 -0.316668   -0.

In [22]:
print_matrix(calculate_gamma(ei_vec_matrix))
# Matrix library has the same problem

   +0.909682     +0.127064     -0.139127     +0.085879     -0.053083     +0.094070     -0.132851     +0.131798     -0.084621     +0.061189
   +0.127064     +0.868215     +0.081502     -0.057554     +0.101260     -0.132851     +0.124608     -0.080151     +0.065566     -0.097659
   -0.139127     +0.081502     +0.949788     +0.096883     -0.137322     +0.131798     -0.080151     +0.058376     -0.093189     +0.131441
   +0.085879     -0.057554     +0.096883     +0.870020     +0.127421     -0.084621     +0.065566     -0.093189     +0.124251     -0.134656
   -0.053083     +0.101260     -0.137322     +0.127421     +0.922720     +0.061189     -0.097659     +0.131441     -0.134656     +0.078688
   +0.094070     -0.132851     +0.131798     -0.084621     +0.061189     +0.909682     +0.127064     -0.139127     +0.085879     -0.053083
   -0.132851     +0.124608     -0.080151     +0.065566     -0.097659     +0.127064     +0.868215     +0.081502     -0.057554     +0.101260
   +0.131798     -0.080151 

# I GAVE UP!!!!
trying to transform and calculate eigenvectors with lapack!

In [23]:
conn = open('temp_size.dat', 'w')
conn.write(str(int(obj.h.shape[0])))
conn.close()
np.savetxt('temp_matrix.dat', obj.h)

In [24]:
obj.h.shape[0]
import subprocess
import os
if not os.path.isfile('calculate_eigenvectors.exe'):
    subprocess.run(['gfortran', 'Calculate_eigenvectors.f90', 'diagmat.f90', 'liblapack.dll', 'libblas.dll', '-o',
                    'calculate_eigenvectors'])
subprocess.run(['calculate_eigenvectors.exe'])

CompletedProcess(args=['calculate_eigenvectors.exe'], returncode=0)

In [25]:
ei_vec = np.loadtxt('temp_ei_vec.dat')
ei_val = np.loadtxt('temp_ei_val.dat')

print_matrix(calculate_gamma(ei_vec))

   +0.900000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000
   +0.100000     +0.900000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000
   -0.100000     +0.100000     +0.900000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000
   +0.100000     -0.100000     +0.100000     +0.900000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     -0.100000
   -0.100000     +0.100000     -0.100000     +0.100000     +0.900000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000
   +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     +0.900000     +0.100000     -0.100000     +0.100000     -0.100000
   -0.100000     +0.100000     -0.100000     +0.100000     -0.100000     +0.100000     +0.900000     +0.100000     -0.100000     +0.100000
   +0.100000     -0.100000 

In [26]:
obj = single_shot.Householder(10, 10, 8)
obj.calculate_one()

calculation with Ns=10, Ne=10, density=1.0


In [42]:
mat1 = np.array(
    [[1, 0, 0],
     [0, 1, 0],
     [0, 0, 1]]
)

mat2 = np.array(
    [[0, 1, 1],
     [1, 0, 1],
     [1, 1, 0]]
)

In [43]:
a, e = np.linalg.eig(mat2)

In [50]:
a

array([-1.,  2., -1.])

In [45]:
e


array([[-0.81649658,  0.57735027,  0.22645541],
       [ 0.40824829,  0.57735027, -0.79259392],
       [ 0.40824829,  0.57735027,  0.56613852]])

In [48]:
c = (e[:,0] + e[:,2])/np.sqrt(2)
mat2 @ c + c

array([0., 0., 0.])

In [49]:
np.dot(e[:,0], e[:,2])

-0.27735009811261446

In [54]:
a2, e2 =np.linalg.eigh(mat2, UPLO='U')
a2

array([-1., -1.,  2.])

In [55]:
e2

array([[-0.71523999,  0.39382538,  0.57735027],
       [ 0.01655721, -0.81632869,  0.57735027],
       [ 0.69868277,  0.42250331,  0.57735027]])

In [57]:
np.dot(e2[:,0], e2[:,1])

-1.3877787807814457e-16

In [61]:
??np.linalg.eigh
